In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.cluster import DBSCAN
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import BaggingClassifier
import xgboost as xgb
from imblearn.over_sampling import SMOTE

In [10]:
xtrain = pd.read_csv("X_train.csv")
xtrain = xtrain.iloc[:, 1:]
xtest = pd.read_csv("X_test.csv")
xtest = xtest.iloc[:, 1:]
ytrain = pd.read_csv("Y_train.csv")

# Remove id from ytrain
ytrain = ytrain.drop('id', axis = 1)


In [11]:
# Bootstrap pipeline with bagging classifier

kf = KFold(n_splits=5, shuffle = True)

c_values = np.array([0.8])
bandwidth = np.array([0.001])
BMAC_means_test = np.array([])
BMAC_stds_test = np.array([])
BMAC_means_train = np.array([])
BMAC_stds_train = np.array([])


for g in bandwidth:
    for c in c_values:
        BMAC_scores_test = np.array([])
        BMAC_scores_train = np.array([])
        for train_index, test_index in kf.split(xtrain.values):

            x_train = xtrain.loc[train_index, :]
            x_test = xtrain.loc[test_index, :]

            y_train = ytrain.loc[train_index, :]
            y_test = ytrain.loc[test_index, :]

            # Handle class imbalance with smote function
            sm = SMOTE(random_state=42)
            x_train, y_train = sm.fit_resample(x_train.values, y_train.values) #x_train and y_train are now arrays

            # Scale the data (should this be done for each bootstrap sample? in that case how)
            scaler = preprocessing.StandardScaler()
            x_train = scaler.fit_transform(x_train)
            #x_train = pd.DataFrame(xtrain_scaled, columns = x_train.columns)
            x_test = scaler.fit_transform(x_test)
            #x_test = pd.DataFrame(xtest_scaled, columns = x_test.columns)

            # Model to fit
            estimator = SVC(C=c, gamma = g)
            #estimator = xgb.XGBClassifier(objective="multi:softprob", random_state=40)
            '''
            bagging = BaggingClassifier(base_estimator = estimator,
                                       n_estimators = 20)
            bagging.fit(x_train, y_train)
            bagging_prediction = bagging.predict(x_test)
            BMAC = balanced_accuracy_score(y_test, bagging_prediction)
            '''
            estimator.fit(x_train, y_train.ravel())
            pred = estimator.predict(x_test)
            BMAC_test = balanced_accuracy_score(y_test, pred)
            BMAC_scores_test = np.append(BMAC_scores_test, BMAC_test)

            pred = estimator.predict(x_train)
            BMAC_train = balanced_accuracy_score(y_train, pred)
            BMAC_scores_train = np.append(BMAC_scores_train, BMAC_train)

        print('mean for current c=', c,'and g=', g, 'is, test:', np.mean(BMAC_scores_test), 'train:',  np.mean(BMAC_scores_train))
    
    
        BMAC_means_test = np.append(BMAC_means_test, np.mean(BMAC_scores_test))
        BMAC_stds_test = np.append(BMAC_stds_test, np.std(BMAC_scores_test))
        BMAC_means_train = np.append(BMAC_means_train, np.mean(BMAC_scores_train))
        BMAC_stds_train = np.append(BMAC_stds_train, np.std(BMAC_scores_train))
    

print(BMAC_means_test)
print(BMAC_stds_test)
print(BMAC_means_train)
print(BMAC_stds_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


mean for current c= 0.3 and g= 1e-05 is, test: 0.5413375035384604 train: 0.6329559923142811


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


mean for current c= 0.5 and g= 1e-05 is, test: 0.5709278045851285 train: 0.6683297961283683


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

mean for current c= 0.8 and g= 0.001 is, test: 0.6694109229271712 train: 0.9827819769568313


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


mean for current c= 1.0 and g= 0.001 is, test: 0.6610353913960721 train: 0.9865693295468576


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: gamma < 0

In [19]:
      
xtrain = pd.read_csv("X_train.csv")
xtrain = xtrain.iloc[:, 1:]
xtest = pd.read_csv("X_test.csv")
xtest = xtest.iloc[:, 1:]
ytrain = pd.read_csv("Y_train.csv")

# Remove id from ytrain
ytrain = ytrain.drop('id', axis = 1)

sm = SMOTE(random_state=42)
xtrain, ytrain = sm.fit_resample(xtrain.values, ytrain.values) #x_train and y_train are now arrays

# Scale the data (should this be done for each bootstrap sample? in that case how)
scaler = preprocessing.StandardScaler()
xtrain = scaler.fit_transform(xtrain)
#x_train = pd.DataFrame(xtrain_scaled, columns = x_train.columns)
xtest = scaler.fit_transform(xtest)
#x_test = pd.DataFrame(xtest_scaled, columns = x_test.columns)

    
    
model = SVC(C=0.8, gamma = 0.001)
model.fit(xtrain, ytrain)
pred = model.predict(xtest)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


NameError: name 'csv' is not defined

In [20]:
index = pd.read_csv("sample.csv")
index['y'] = pred

index.to_csv("svm_tuned_for_loops.csv")